# <center>Đồ án 3: Linear Regression</center>


# Thông tin sinh viên

- Họ và tên: Lê Phước Phát
- MSSV: 22127322
- Lớp: 22CLC10


# Import


In [1]:
import pandas as pd
import numpy as np

# Import thêm dữ thư viện nếu cần
import plotly.express as px
import plotly.graph_objects as go


# Đọc dữ liệu


In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv(
    "train.csv"
)  # Đọc tệp train.csv vào biến train, dưới dạng một DataFrame của pandas.
test = pd.read_csv(
    "test.csv"
)  # Đọc tệp test.csv vào biến test, dưới dạng một DataFrame của pandas.

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]  # Dataframe (chứa 5 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]  # Series    (chứa 1 giá trị mục tiêu huấn luyện)

X_test = test.iloc[:, :-1]  # Dataframe (chứa 5 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]  # Series    (chứa 1 giá trị mục tiêu kiểm tra)


# Cài đặt hàm


## 1. Linear Regression using Ordinary Least Squares (OLS)


### 1.1 Preprocessing data


In [4]:
def preprocess(x):
    """
    This function is used to preprocess the data. It adds a column of ones to the input data and squares the input data.

    Parameters
    ----------
    x : np.array
        Input data

    Returns
    -------
    X : np.array
        Preprocessed input data
    """
    X = np.hstack([np.ones((x.shape[0], 1)), x])  # Thêm cột 1 vào X để tính b

    return X


In [5]:
def standard_scaler(X):
    """
    Chuẩn hóa dữ liệu bằng StandardScaler
    Parameters:
    -----------
    X : np.array
        Dữ liệu đầu vào

    Returns:
    --------
    X_scaled : np.array
        Dữ liệu đã được chuẩn hóa
    mean : np.array
        Giá trị trung bình của mỗi đặc trưng
    std : np.array
        Độ lệch chuẩn của mỗi đặc trưng
    """
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_scaled = (X - mean) / std
    return X_scaled, mean, std


### 1.2 Linear Regression


#### Giới thiệu

- **Hồi quy tuyến tính (Linear Regression)** là một kỹ thuật phân tích thống kê được sử dụng để mô hình hóa mối quan hệ giữa một hoặc nhiều biến đầu vào (biến độc lập) và một biến đầu ra (biến phụ thuộc).
- **Mục tiêu** của hồi quy tuyến tính là tìm một đường thẳng (hoặc một hyperplane trong không gian đa chiều) tốt nhất mà dữ liệu được "khớp" với nó.

#### Công thức

- Trong hồi quy tuyến tính sử dụng phương pháp Bình phương Tối thiểu Thông thường (Ordinary Least Square - OLS), mục tiêu là tìm vector tham số tối ưu $w$ sao cho tổng phương của các sai số giữa giá trị dự đoán và giá trị thực tế là nhỏ nhất.
  $$w = (X^TX)^{-1}X^Ty$$
- Trong đó:
  - $w$: vector chứa các hệ số hồi quy tối ưu (vector trọng số).
  - $X$: ma trận chứa các giá trị của các biến độc lập (các đặc trưng, features) được sử dụng để dự đoán biến phụ thuộc $y$.
  - $y$: vector chứa các giá trị mục tiêu hoặc biến phụ thuộc mà mô hình cần dự đoán, là một vector một chiều với kích thước $n×1$, trong đó $n$ là số lượng mẫu (giống với số lượng hàng của $X$).

$Xw \approx y$ hay $ Xw = y$ ($y$ được gọi là đường hồi quy (regression line))


In [6]:
class OLSLinearRegression:
    def fit(self, X, y):
        """
        This function is used to fit the model to the data. It uses the Ordinary Least Squares method to find the optimal parameters.

        Parameters
        ----------
        X : np.array
            Input data
        y : np.array
            Output data

        Returns
        -------
        self : object
            Returns the instance of the class

        """

        X_pinv = np.linalg.inv(X.T @ X) @ X.T
        self.w = X_pinv @ y

        return self

    def get_params(self):
        """
        This function is used to get the parameters of the model.

        Returns
        -------
        self.w : np.array
            Optimal parameters (column vector)
        """

        return self.w

    def predict(self, X):
        """
        This function is used to predict the output of the model.

        Parameters
        ----------
        X : np.array
            Input data

        Returns
        -------
        X @ self.w : np.array
            Predicted output
        """

        return X @ self.w


## 2. Mean Absolute Error (MAE)


MAE được dùng để ước lượng **trung bình của sai số (độ lỗi) tuyệt đối**, được tính bằng công thức:
$$MAE = \frac{1}{n} \sum_{n}^{i = 1}|y_i - \hat{y_i}|$$

Trong đó:

- $n$: số lượng mẫu quan sát
- $y_i$: giá trị mục tiêu của mẫu thứ $i$, là mảng numpy chứa các giá trị thực tế (ground truth) mà ta muốn so sánh với giá trị dự đoán.
- $\hat{y_i}$: giá trị mục tiêu của mẫu thứ $i$ được dự đoán từ mô hình hồi quy tuyến tính, là mảng numpy chứa các giá trị dự đoán từ mô hình.


In [7]:
def mae(y, y_hat):
    """
    This function is used to calculate the mean absolute error (MAE).

    Parameters
    ----------
    y : np.array
        Output data
    y_hat : np.array
        Predicted output data

    Returns
    -------
    np.mean(np.abs(y.ravel() - y_hat.ravel())) : float
        Mean absolute error
    """
    return np.mean(np.abs(y.ravel() - y_hat.ravel()))


_Chú thích: Cần có docstrings cho các hàm._


# Yêu cầu 1: Phân tích khám phá dữ liệu (1 điểm)


## 1.1. Thống kê và mô tả dữ liệu các đặc trưng


In [8]:
# Phân tích khám phá dữ liệu thông qua thống kê và các biểu đồ
# Chỉ được phân tích trên tập huấn luyện (train dataset)

print("Thông tin tổng quan về dữ liệu:")
print(train.info())


Thông tin tổng quan về dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     9000 non-null   int64  
 1   Previous Scores                   9000 non-null   int64  
 2   Extracurricular Activities        9000 non-null   int64  
 3   Sleep Hours                       9000 non-null   int64  
 4   Sample Question Papers Practiced  9000 non-null   int64  
 5   Performance Index                 9000 non-null   float64
dtypes: float64(1), int64(5)
memory usage: 422.0 KB
None


- Nhìn vào thông tin tổng quan về dữ liệu, ta có thể thấy số lượng mẫu (entries) là 9000 mẫu (từ 0 đến 8999), gồm 6 cột, trong đó:
  - Có **5** cột đầu tiên (Hour Studied, Previous Scores, Extracurricular Activities, Sleep Hours, Sample Question Papers Practiced) với kiểu dữ liệu **int64**, đại diện cho các biến số nguyên.
  - Có **1** cột cuối cùng (Performance Index) với kiểu dữ liệu **float64**, đại diện cho một biến số thực (biến liên tục).
- Các cột dữ liệu:
  - **Hours Studied**: Tổng số giờ học của mỗi sinh viên.
  - **Previous Scores**: Điểm số học sinh đạt được trong các bài kiểm tra trước đó.
  - **Extracurricular Activities**: Sinh viên có tham gia hoạt động ngoại khóa không (Có hoặc Không).
  - **Sleep Hours**: Số giờ ngủ trung bình mỗi ngày của sinh viên.
  - **Sample Question Papers Practiced**: Số bài kiểm tra mẫu mà học sinh đã luyện tập.
  - **Performance Index**: Thước đo thành tích tổng thể cho mỗi sinh viên. Chỉ số thể hiện thành tích học tập, nằm trong đoạn [10, 100]. Chỉ số này tỉ lệ thuận với thành tích.
- Tất cả các cột đều có 9000 **giá trị không thiếu (non-null)**, điều này có nghĩa là không có giá trị nào bị thiếu trong tập dữ liệu này.


In [9]:
print("\n5 hàng đầu tiên của dữ liệu:")
print(train.head())



5 hàng đầu tiên của dữ liệu:
   Hours Studied  Previous Scores  Extracurricular Activities  Sleep Hours  \
0              7               77                           0            5   
1              8               90                           1            4   
2              9               83                           1            6   
3              4               52                           0            9   
4              4               82                           1            8   

   Sample Question Papers Practiced  Performance Index  
0                                 2               69.0  
1                                 1               84.0  
2                                 3               82.0  
3                                 5               38.0  
4                                 6               68.0  


- Đây là 5 hàng đầu tiên của bộ dữ liệu huấn luyện (training set).


In [10]:
print("\nKiểm tra các giá trị thiếu:")
print(train.isnull().sum())



Kiểm tra các giá trị thiếu:
Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64


- Chúng ta có thể thấy các dữ liệu đã được tiền xử lý kỹ càng, không có giá trị nào rỗng hay bị thiếu dữ liệu $\to$ dữ liệu đã được làm sạch.


In [11]:
# Thống kê mô tả dữ liệu
print("\nThống kê mô tả dữ liệu:")
print(train.describe())



Thống kê mô tả dữ liệu:
       Hours Studied  Previous Scores  Extracurricular Activities  \
count    9000.000000      9000.000000                 9000.000000   
mean        4.976444        69.396111                    0.493667   
std         2.594647        17.369957                    0.499988   
min         1.000000        40.000000                    0.000000   
25%         3.000000        54.000000                    0.000000   
50%         5.000000        69.000000                    0.000000   
75%         7.000000        85.000000                    1.000000   
max         9.000000        99.000000                    1.000000   

       Sleep Hours  Sample Question Papers Practiced  Performance Index  
count  9000.000000                       9000.000000        9000.000000  
mean      6.535556                          4.590889          55.136333  
std       1.695533                          2.864570          19.187669  
min       4.000000                          0.000000     

- Nhận xét chung
  - Hầu hết các biến đều có phân phối rộng, với giá trị min và max khác biệt, đặc biệt là ở **Performance Index**, **Previous Scores**, và **Hours Studied**.
  - Các biến như **Hours Studied**, **Sleep Hours**, và **Sample Question Papers Practiced** có độ phân tán cao, được thể hiện qua độ lệch chuẩn.
  - Các biến như **Performance Index** và **Previous Scores** có thể có phân phối lệch hoặc có các giá trị cực trị.
  - **Extracurricular Activities** có tính chất nhị phân (0 hoặc 1), trong đó phần lớn giá trị là 0.


In [12]:
# Kiểm tra phân phối của các đặc trưng
print("\nPhân phối của các đặc trưng:")
for column in train.columns:
    print(f"\nPhân phối của {column}:")
    print(train[column].value_counts())



Phân phối của các đặc trưng:

Phân phối của Hours Studied:
Hours Studied
1    1062
7    1012
6    1011
9    1000
3    1000
5     986
2     979
4     978
8     972
Name: count, dtype: int64

Phân phối của Previous Scores:
Previous Scores
54    196
87    182
56    180
62    168
53    167
89    165
52    163
77    163
90    162
47    161
83    160
58    159
84    158
65    158
41    158
95    157
66    157
49    157
60    156
67    156
97    155
57    155
92    154
42    154
43    154
93    154
40    153
48    153
79    153
91    152
75    151
88    150
46    150
44    149
70    149
73    149
61    148
59    147
85    146
86    146
96    145
78    145
63    144
81    142
80    142
98    142
71    141
45    140
68    138
99    137
94    136
72    136
69    135
64    130
76    130
82    127
55    127
74    125
51    121
50    112
Name: count, dtype: int64

Phân phối của Extracurricular Activities:
Extracurricular Activities
0    4557
1    4443
Name: count, dtype: int64

Phân phối của Sleep

- Các biến **Hours Studied**, **Sleep Hours**, và **Sample Question Papers Practiced** đều có phân phối khá đồng đều, không có giá trị nào chiếm ưu thế quá lớn.
- Các biến **Previous Scores** và **Performance Index** có sự phân bố rộng với nhiều giá trị khác nhau, điều này cho thấy dữ liệu có sự đa dạng cao.


In [19]:
# Kiểm tra mối quan hệ giữa các đặc trưng
print("\nMối quan hệ giữa các đặc trưng:")
correlation_matrix = train.corr()
print(correlation_matrix)



Mối quan hệ giữa các đặc trưng:
                                  Hours Studied  Previous Scores  \
Hours Studied                          1.000000        -0.018463   
Previous Scores                       -0.018463         1.000000   
Extracurricular Activities             0.004511         0.009533   
Sleep Hours                           -0.000694         0.002802   
Sample Question Papers Practiced       0.015852         0.006417   
Performance Index                      0.369148         0.914775   

                                  Extracurricular Activities  Sleep Hours  \
Hours Studied                                       0.004511    -0.000694   
Previous Scores                                     0.009533     0.002802   
Extracurricular Activities                          1.000000    -0.020773   
Sleep Hours                                        -0.020773     1.000000   
Sample Question Papers Practiced                    0.008199     0.005054   
Performance Index           

## 1.2. Sử dụng biểu đồ để phân tích/quan sát các đặc trưng


In [13]:
# Lấy các đặc trưng phù hợp để vẽ biểu đồ tròn
features_to_plot = [
    "Extracurricular Activities",
    "Sleep Hours",
    "Sample Question Papers Practiced",
]

# Tạo biểu đồ tròn cho mỗi đặc trưng
for feature in features_to_plot:
    # Tính toán số lượng các giá trị phân loại
    value_counts = train[feature].value_counts()

    # Tạo biểu đồ tròn
    fig = go.Figure(
        data=[go.Pie(labels=value_counts.index, values=value_counts.values, hole=0.0)]
    )

    # Thay đổi tiêu đề và các tùy chọn hiển thị
    fig.update_layout(
        title=f"Distribution of {feature}", showlegend=True, legend_title="Categories"
    )

    # Hiển thị biểu đồ
    fig.show()


In [14]:
# Lấy các đặc trưng số để vẽ biểu đồ histogram
numeric_features = [
    "Hours Studied",
    "Previous Scores",
    "Sleep Hours",
    "Sample Question Papers Practiced",
]

# Tạo biểu đồ histogram cho mỗi đặc trưng
for feature in numeric_features:
    # Tạo biểu đồ histogram
    fig = go.Figure(data=[go.Histogram(x=train[feature], nbinsx=30)])

    # Thay đổi tiêu đề và các tùy chọn hiển thị
    fig.update_layout(
        title=f"Distribution of {feature}",
        xaxis_title=feature,
        yaxis_title="Frequency",
        bargap=0.2,  # Khoảng cách giữa các thanh trong histogram
    )

    # Hiển thị biểu đồ
    fig.show()


In [15]:
# Chọn cặp đặc trưng để vẽ biểu đồ phân tán
feature1 = "Hours Studied"
feature2 = "Performance Index"

# Tạo biểu đồ phân tán với đường xu hướng
fig = px.scatter(
    train,
    x=feature1,
    y=feature2,
    trendline="ols",  # Thêm đường xu hướng hồi quy tuyến tính
    title=f"Scatter Plot giữa {feature1} và {feature2}",
    labels={feature1: feature1, feature2: feature2},
)

# Hiển thị biểu đồ
fig.show()


In [16]:
# Chọn cặp đặc trưng để vẽ biểu đồ phân tán
feature1 = "Previous Scores"
feature2 = "Performance Index"

# Tạo biểu đồ phân tán với đường xu hướng
fig = px.scatter(
    train,
    x=feature1,
    y=feature2,
    trendline="ols",  # Thêm đường xu hướng hồi quy tuyến tính
    title=f"Scatter Plot giữa {feature1} và {feature2}",
    labels={feature1: feature1, feature2: feature2},
)

# Hiển thị biểu đồ
fig.show()


In [17]:
# Chọn cặp đặc trưng để vẽ biểu đồ phân tán
feature1 = "Sleep Hours"
feature2 = "Performance Index"

# Tạo biểu đồ phân tán với đường xu hướng
fig = px.scatter(
    train,
    x=feature1,
    y=feature2,
    trendline="ols",  # Thêm đường xu hướng hồi quy tuyến tính
    title=f"Scatter Plot giữa {feature1} và {feature2}",
    labels={feature1: feature1, feature2: feature2},
)

# Hiển thị biểu đồ
fig.show()


In [18]:
# 2. Biểu đồ hộp (box plot)
print("Biểu đồ hộp (box plot) cho các đặc trưng số:")
for column in train.select_dtypes(include=["int64", "float64"]).columns:
    fig = px.box(train, y=column, title=f"Box plot của {column}")
    fig.show()


Biểu đồ hộp (box plot) cho các đặc trưng số:


In [19]:
# 4. Biểu đồ heatmap cho ma trận tương quan

# Giả sử 'train' là DataFrame của bạn
corr_matrix = train.corr()

heatmap = go.Heatmap(
    z=corr_matrix.values,  # Dữ liệu ma trận tương quan
    x=corr_matrix.columns,  # Tên các đặc trưng (cột)
    y=corr_matrix.columns,  # Tên các đặc trưng (dòng)
    colorscale="RdBu",  # Màu sắc từ danh sách màu hợp lệ của Plotly
    zmin=-1,  # Giá trị tối thiểu cho thang đo màu
    zmax=1,  # Giá trị tối đa cho thang đo màu
    text=np.round(corr_matrix.values, 4),  # Hiển thị giá trị tương quan trên heatmap
    texttemplate="%{text}",  # Hiển thị giá trị tương quan
    hoverinfo="text",  # Hiển thị giá trị khi di chuột qua
)

# Tạo figure và thêm heatmap vào
fig = go.Figure(data=heatmap)

# Thêm tiêu đề cho biểu đồ
fig.update_layout(title="Heatmap of Correlation Matrix")

# Hiển thị biểu đồ
fig.show()


# Yêu cầu 2a: Xây dựng mô hình sử dụng toàn bộ 5 đặc trưng đề bài cung cấp (2 điểm)


In [20]:
# Phần code cho yêu cầu 2a
def process_2a(X_train, y_train, X_test):
    """
    This function preprocesses the training and test data, fits an Ordinary Least Squares (OLS)
    linear regression model to the training data, and returns the model coefficients and
    predictions on the test data.

    Args:
        X_train (np.ndarray or pd.DataFrame): The feature matrix for the training data.
        y_train (np.ndarray or pd.Series): The target variable for the training data.
        X_test (np.ndarray or pd.DataFrame): The feature matrix for the test data.

    Returns:
        tuple: A tuple containing:
            - coefficients (np.ndarray): The coefficients (weights) of the trained OLS model.
            - y_hat (np.ndarray): The predicted target values for the test data.
    """
    # Preprocess the training data
    X_train = preprocess(X_train)
    # Preprocess the test data
    X_test = preprocess(X_test)
    # Initialize the OLS linear regression model
    model = OLSLinearRegression()
    # Fit the model to the training data
    model.fit(X_train, y_train)
    # Retrieve the coefficients from the trained model
    coefficients = model.get_params()
    # Predict the target values for the test data
    y_hat = model.predict(X_test)
    # Return the coefficients and the predicted values
    return coefficients, y_hat


In [32]:
# Extract features (X) and target (y) from the training data as numpy arrays
X_train = train.iloc[
    :, :-1
].values  # Select all columns except the last one as features for training
y_train = train.iloc[
    :, -1
].values  # Select the last column as the target variable for training
# Extract features (X) and target (y) from the test data as numpy arrays
X_test = test.iloc[
    :, :-1
].values  # Select all columns except the last one as features for testing
y_test = test.iloc[
    :, -1
].values  # Select the last column as the target variable for testing
print(X_train.shape)
# Returns the model coefficients and predictions
coefs, y_hat = process_2a(X_train, y_train, X_test)
print("weights: ", coefs)  # Print the calculated weights (coefficients) of the model
print(
    "y_pred: ", y_hat.shape
)  # Print the shape of the predicted values array to confirm the number of predictions made


(9000, 5)
weights:  [-33.96928368   2.85202007   1.01786957   0.60428174   0.47356583
   0.19237624]
y_pred:  (1000,)


In [22]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
error = mae(
    y_test, y_hat
)  # Calculate the MAE Error between the true target values (y_test) and the predicted values (y_hat)
print("MAE Error = ", error)


MAE Error =  1.595648688476289


In [23]:
# Plotly scatter plot for the true vs. predicted values
fig = px.scatter(
    x=y_test,
    y=y_hat,
    labels={"x": "True Performance Index", "y": "Predicted Performance Index"},
    title="True vs. Predicted Performance Index",
)
# Add a diagonal line (perfect prediction line)
fig.add_shape(
    type="line",
    line=dict(color="red", width=4),
    x0=min(y_test),
    y0=min(y_test),
    x1=max(y_test),
    y1=max(y_test),
)


Công thức hồi quy, phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012

**$$\text{Student Performance} = -33.969 + 2.852x_1 + 1.018x_2 + 0.604x_3 + 0.474x_4 + 0.192x_5$$**

Trong đó:

- $x_1$: đặc trưng **Hours Studied** chỉ tổng số giờ học của mỗi học sinh.
- $x_2$: đặc trưng **Previous Scores** chỉ điểm số học sinh đạt được trong các bài kiểm tra trước đó.
- $x_3$: đặc trưng **Extracurricular Activities** cho biết sinh viên có tham gia hoạt động ngoại khóa không.
- $x_4$: đặc trưng **Sleep Hours** chỉ số giờ ngủ trung bình mỗi ngày của sinh viên.
- $x_5$: đặc trưng **Sample Question Papers Practiced** cho biết số bài kiểm tra mẫu mà học sinh đã luyện tập.

Ước lượng trung bình của sai số (độ lỗi) tuyệt đối của mô hình trên trên tập test là: **MAE** $\approx 1.596$


# Yêu cầu 2b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)


Lưu ý: Khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng


In [24]:
def shuffle_and_split_folds(X, y, k=5):
    """
    Shuffles the data and splits it into k folds for cross-validation.

    Parameters:
    - X: np.ndarray
        The features of the dataset.
    - y: np.ndarray
        The target values corresponding to the features.
    - k: int, optional, default=5
        The number of folds for cross-validation.

    Returns:
    - folds_X: list of np.ndarray
        A list containing the feature data for each fold.
    - folds_y: list of np.ndarray
        A list containing the target data for each fold.
    """
    n = X.shape[0]  # Number of samples in the dataset
    indices = np.arange(n)  # Create an array of indices for all samples
    np.random.shuffle(indices)  # Shuffle the indices to randomize the data

    # Shuffle the data according to the shuffled indices
    X_shuffle = X[indices]
    y_shuffle = y[indices]

    # Calculate the size of each fold
    fold_size = n // k

    # Split the shuffled data into k folds
    folds_X = [X_shuffle[i * fold_size : (i + 1) * fold_size] for i in range(k)]
    folds_y = [y_shuffle[i * fold_size : (i + 1) * fold_size] for i in range(k)]

    return folds_X, folds_y


In [25]:
# Phần code cho yêu cầu 2b
# Tìm ra đặc trưng tốt nhất (trong 5 đặc trưng)
# In ra các kết quả cross-validation như yêu cầu
def finding_best_feature(X_train, y_train, features, k=5):
    """
    Performs k-fold cross-validation for each feature in the dataset to evaluate the
    mean absolute error (MAE) using a simple linear regression model. The function
    returns the MAE for each feature, allowing identification of the feature that
    yields the lowest MAE.

    Parameters:
    - X_train: np.ndarray
        The training data containing the features.
    - y_train: np.ndarray
        The target values corresponding to the training data.
    - features: pd.Index or list-like
        The names or indices of the features in X_train.
    - k: int, optional, default=5
        The number of folds for cross-validation.

    Returns:
    - mae_results: dict
        A dictionary where keys are feature names and values are the average MAE
        across k folds.
    """
    mae_results = {}  # Dictionary to store MAE for each feature

    # Shuffle and split data into folds
    folds_X, folds_y = shuffle_and_split_folds(X_train, y_train, k=k)

    # Iterate through each feature for cross-validation
    for feature in features:
        mae_list = []  # Reset MAE list for each feature

        print(f"Performing cross-validation for feature: {feature}")

        # Perform k-fold cross-validation
        for i in range(k):
            # Create training and validation sets for the current fold
            X_val_fold = folds_X[i]
            y_val_fold = folds_y[i]
            X_train_folds = np.concatenate(
                [folds_X[j] for j in range(k) if j != i], axis=0
            )
            y_train_folds = np.concatenate(
                [folds_y[j] for j in range(k) if j != i], axis=0
            )

            # Extract data for the current feature
            X_train_fe = X_train_folds[:, features.get_loc(feature)].reshape(-1, 1)
            y_train_fe = y_train_folds

            X_val_fe = X_val_fold[:, features.get_loc(feature)].reshape(-1, 1)
            y_val_fe = y_val_fold

            # Preprocess data (if necessary)
            X_train_fe = preprocess(X_train_fe)
            X_val_fe = preprocess(X_val_fe)

            # Train the model on the training data
            model = OLSLinearRegression()  # Initialize the linear regression model
            model.fit(X_train_fe, y_train_fe)  # Train the model
            y_pred_val_fe = model.predict(X_val_fe)  # Predict on validation data

            # Calculate MAE for this fold
            mae_error = mae(y_val_fe, y_pred_val_fe)
            mae_list.append(mae_error)  # Store MAE for this fold

        # Calculate the average MAE for the current feature
        mae_mean_fe = np.mean(mae_list)
        mae_results[feature] = mae_mean_fe  # Store the result
        print(f"MAE for feature {feature}: {mae_mean_fe}")

    # Identify the feature with the lowest MAE
    best_feature = min(mae_results, key=mae_results.get)
    best_mae = mae_results[best_feature]
    return best_feature, best_mae


In [26]:
def train_best_feature_model(X_best_feature, y_train):
    """
    Train a linear regression model using the best feature.

    This function takes in the training data corresponding to the best-selected feature
    and the associated target values, then trains an OLS linear regression model.

    Args:
        X_best_feature (numpy.ndarray or pandas.DataFrame): The training feature data
            corresponding to the best feature (a single column of data).
        y_train (numpy.ndarray or pandas.Series): The target data corresponding
            to the training feature data.

    Returns:
        best_feature_model (OLSLinearRegression): The trained linear regression model.
        best_feature_params (dict): The parameters of the trained model, including weights
            and intercept.
    """
    # Initialize the linear regression model
    best_feature_model = OLSLinearRegression()
    # Train the model using the best feature data and the target data
    best_feature_model.fit(X_best_feature, y_train)
    # Retrieve the model parameters after training (weights and intercept)
    best_feature_params = best_feature_model.get_params()
    # Return the trained model and its parameters
    return best_feature_model, best_feature_params


In [27]:
# Prepare the data
X_train = train.iloc[:, :-1].values  # Training Features Values
y_train = train.iloc[:, -1].values  # Training Target Values
X_test = test.iloc[:, :-1].values  # Test Features
y_test = test.iloc[:, -1].values  # Test Target
features = train.columns[:-1]  # List of feature names

# Perform cross-validation to evaluate features
best_feature, best_mae = finding_best_feature(X_train, y_train, features, k=5)
print(f"\nThe best feature is: {best_feature} with MAE: {best_mae}")

best_feature_index = features.get_loc(best_feature)

X_best_feature = X_train[:, best_feature_index].reshape(-1, 1)
X_best_feature = preprocess(X_best_feature)

best_feature_model, best_feature_params = train_best_feature_model(
    X_best_feature, y_train
)

print("=================")
print("WEIGHTS")
print(f"[w0]: {best_feature_params[0]} (intercept)")
print(f"[w1]: {best_feature_params[1]}")


Performing cross-validation for feature: Hours Studied
MAE for feature Hours Studied: 15.44914723022427
Performing cross-validation for feature: Previous Scores
MAE for feature Previous Scores: 6.618073861271869
Performing cross-validation for feature: Extracurricular Activities
MAE for feature Extracurricular Activities: 16.195293828370936
Performing cross-validation for feature: Sleep Hours
MAE for feature Sleep Hours: 16.186225059070345
Performing cross-validation for feature: Sample Question Papers Practiced
MAE for feature Sample Question Papers Practiced: 16.183961056550412

The best feature is: Previous Scores with MAE: 6.618073861271869
WEIGHTS
[w0]: -14.988645779326767 (intercept)
[w1]: 1.0105030093167888


In [28]:
X_test_best_feature = X_test[:, best_feature_index].reshape(-1, 1)
X_test_best_feature = preprocess(X_test_best_feature)
y_hat_best_feature = best_feature_model.predict(X_test_best_feature)


In [29]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
error_best_feature = mae(y_test, y_hat_best_feature)
print("MAE Error = ", error_best_feature)


MAE Error =  6.5442772934525015


In [30]:
# Plotly scatter plot for the true vs. predicted values
fig = px.scatter(
    x=y_test,
    y=y_hat_best_feature,
    labels={"x": "True Performance Index", "y": "Predicted Performance Index"},
    title="True vs. Predicted Performance Index",
)
# Add a diagonal line (perfect prediction line)
fig.add_shape(
    type="line",
    line=dict(color="red", width=4),
    x0=min(y_test),
    y0=min(y_test),
    x1=max(y_test),
    y1=max(y_test),
)


Công thức hồi quy (dựa trên mô hình đặc trưng tốt nhất), phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012

$$\text{Student Performance} = -14.989 + 1.011x$$

Trong đó:

- $x$: đặc trưng **Previous Score** cho biết điểm số học sinh đạt được trong các bài kiểm tra trước đó.

Ước lượng trung bình của sai số (độ lỗi) tuyệt đối của mô hình trên trên tập test là: **MAE** $\approx 6.544$


# Yêu cầu 2c: Sinh viên tự xây dựng/thiết kế mô hình, tìm mô hình cho kết quả tốt nhất (2 điểm)


## Xây dựng/Thiết kế mô hình


In [68]:
# Trình bày toàn bộ code liên quan đến việc thiết kế mô hình
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]

X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Định nghĩa các mô hình với các đặc trưng khác nhau
models = {
    "Model 1: Interaction Features": lambda X_train: preprocess(
        pd.DataFrame(X_train, columns=train.columns[:-1])[["Hours Studied"]].values
        * pd.DataFrame(X_train, columns=train.columns[:-1])[
            ["Previous Scores"]
        ].values  # Tạo đặc trưng tương tác giữa Hours Studied và Previous Scores
    ),
    "Model 2: Squared Features": lambda X_train: preprocess(
        pd.DataFrame(X_train, columns=train.columns[:-1])[
            [
                "Hours Studied",
                "Previous Scores",
                "Sleep Hours",
                "Sample Question Papers Practiced",
            ]
        ].values
        ** 2  # Sử dụng bình phương của các đặc trưng Hours Studied và Sleep Hours
    ),
    "Model 3: 3 Standard Scaler Features": lambda X_train: preprocess(
        pd.DataFrame(X_train, columns=train.columns[:-1])[
            ["Hours Studied", "Previous Scores", "Sleep Hours"]
        ].values
    ),
}


## Tìm mô hình cho kết quả tốt nhất


Lưu ý: Khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế


In [69]:
# Phần code cho yêu cầu 2c
# Tìm ra mô hình tốt nhất (trong m mô hình mà sinh viên tự thiết kế)
# In ra các kết quả cross-validation như yêu cầu
def finding_best_m_model(X_train, y_train, models, k=5):
    mae_results = {}

    # Shuffle and split data into folds
    folds_X, folds_y = shuffle_and_split_folds(X_train, y_train, k=k)

    for model_name, model_fn in models.items():
        mae_list = []
        print(f"Performing cross-validation for model: {model_name}")
        for i in range(k):

            X_val_fold = folds_X[i]
            y_val_fold = folds_y[i]
            X_train_folds = np.concatenate(
                [folds_X[j] for j in range(k) if j != i], axis=0
            )
            y_train_folds = np.concatenate(
                [folds_y[j] for j in range(k) if j != i], axis=0
            )

            if model_name == "Model 3: 3 Standard Scaler Features":
                # Chuẩn hóa dữ liệu
                X_train_model, mean, std = standard_scaler(X_train_folds)
                X_val_model = (X_val_fold - mean) / std

                X_train_model = model_fn(X_train_model)
                y_train_model = y_train_folds

                X_val_model = model_fn(X_val_model)
                y_val_model = y_val_fold
            else:
                X_train_model = model_fn(X_train_folds)
                y_train_model = y_train_folds

                X_val_model = model_fn(X_val_fold)
                y_val_model = y_val_fold

            model = OLSLinearRegression()
            model.fit(X_train_model, y_train_model)
            y_pred_val_model = model.predict(X_val_model)

            mae_error = mae(y_val_model, y_pred_val_model)
            mae_list.append(mae_error)

        mae_results[model_name] = np.mean(mae_list)
        print(f"MAE for model {model_name}: {mae_results[model_name]}")

    best_model = min(mae_results, key=mae_results.get)
    best_mae_model = mae_results[best_model]
    return best_model, best_mae_model


In [70]:
best_model, best_mae_model = finding_best_m_model(
    X_train.values, y_train.values, models, k=5
)
print(f"\nBest Model: {best_model} with MAE = {best_mae_model}")


Performing cross-validation for model: Model 1: Interaction Features
MAE for model Model 1: Interaction Features: 11.081988100334428
Performing cross-validation for model: Model 2: Squared Features
MAE for model Model 2: Squared Features: 2.657417600151048
Performing cross-validation for model: Model 3: 3 Standard Scaler Features
MAE for model Model 3: 3 Standard Scaler Features: 1.7015971636536862

Best Model: Model 3: 3 Standard Scaler Features with MAE = 1.7015971636536862


In [71]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
X_best_model = models[best_model](train)
my_best_model = OLSLinearRegression()
my_best_model.fit(X_best_model, y_train.values)

X_test_best_model = models[best_model](test)
print(X_test_best_model)
y_pred_test = my_best_model.predict(X_test_best_model)


[[ 1.  7. 74.  8.]
 [ 1.  6. 89.  8.]
 [ 1.  3. 79.  8.]
 ...
 [ 1.  9. 44.  9.]
 [ 1.  3. 41.  5.]
 [ 1.  8. 93.  6.]]


In [72]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
mae_test = mae(y_test.values, y_pred_test)
print(f"MAE on test set for the best model: {mae_test}")


MAE on test set for the best model: 1.6942928316021288


In [73]:
# Lấy trọng số và tạo công thức hồi quy
weights_best = np.round(my_best_model.get_params(), 3)
b = weights_best[0]
w1 = weights_best[1]
w2 = weights_best[2]
w3 = weights_best[3]
weights = weights_best[1:]

print("b: ", b)
print("[w1]: ", w1)
print("[w2]: ", w2)
print("[w3]: ", w3)
print("weights: ", weights)


b:  -32.82
[w1]:  2.856
[w2]:  1.018
[w3]:  0.472
weights:  [2.856 1.018 0.472]


Công thức hồi quy (dựa trên mô hình tốt nhất mà sinh viên tự xây dựng/thiết kế), phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012

$$\text{Student Performance} = -32.82 + 2.856x_1 + 1.018x_2 + 0.472x_3$$

Trong đó:

- $x_1$: đặc trưng **Hours Studied** chỉ tổng số giờ học của mỗi học sinh.
- $x_2$: đặc trưng **Previous Scores** chỉ điểm số học sinh đạt được trong các bài kiểm tra trước đó.
- $x_3$: đặc trưng **Sleep Hours** chỉ số giờ ngủ trung bình mỗi ngày của sinh viên.

Ước lượng trung bình của sai số (độ lỗi) tuyệt đối của mô hình trên trên tập test là: **MAE** $\approx 1.694$
